In [17]:
%matplotlib inline
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split as train
from sklearn.model_selection import StratifiedKFold as strat
from sklearn.model_selection import cross_validate as cross
from collections import Counter

# Bears 
+ Download the bears dataset and its description from Moodle
+ Load the dataset as a pandas DataFrame


In [6]:
df = pd.read_csv("bears.csv",sep=";")
df.head()

,AGE,MONTH,SEX,HEADLEN,HEADWTH,NECK,LENGTH,CHEST,WEIGHT
0,19,7,1,11.0,5.5,16.0,53.0,26.0,80
1,55,7,1,16.5,9.0,28.0,67.5,45.0,344
2,81,9,1,15.5,8.0,31.0,72.0,54.0,416
3,115,7,1,17.0,10.0,31.5,72.0,49.0,348
4,104,8,2,15.5,6.5,22.0,62.0,35.0,166


## Classification
The goal of this section is to train Machine Learning classifiers to predict the bears' sex from the other attributes
+ Extract the sex column from the dataset, in order to use it as a class membership vector.
+ Print the statistical descriptors for each column of the dataset
+ What can you say about the features' scales?


In [9]:
sex = df['SEX']
sex.head()

df.describe()    

,AGE,MONTH,SEX,HEADLEN,HEADWTH,NECK,LENGTH,CHEST,WEIGHT
count,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000
mean,43.518519,8.407407,1.351852,12.953704,6.194444,20.555556,58.616667,35.662963,182.888889
std,33.720684,2.004886,0.482032,2.144032,1.512266,5.640710,10.700868,9.351729,121.801209
min,8.000000,4.000000,1.000000,9.000000,4.000000,10.000000,36.000000,19.000000,26.000000
25%,17.000000,8.000000,1.000000,11.500000,5.000000,16.625000,50.500000,29.000000,87.000000
50%,34.000000,9.000000,1.000000,13.000000,6.000000,20.000000,60.750000,34.000000,150.000000
75%,57.750000,10.000000,2.000000,14.500000,7.000000,24.000000,66.125000,43.500000,232.000000
max,177.000000,11.000000,2.000000,17.000000,10.000000,31.500000,76.500000,55.000000,514.000000


You can notice that the scales of the different attributes are quite different - can be problemactic for KNN means

+ Apply a Z-score standardization to the dataset, and create another one.
+ Compute the same previous statistics

In [11]:
df_zscore1 = (df - df.mean()) / df.std()
df_zscore1.describe()

,AGE,MONTH,SEX,HEADLEN,HEADWTH,NECK,LENGTH,CHEST,WEIGHT
count,5.400000e+01,5.400000e+01,5.400000e+01,5.400000e+01,5.400000e+01,5.400000e+01,5.400000e+01,5.400000e+01,5.400000e+01
mean,-2.261565e-17,4.605370e-16,-5.345518e-17,-2.605940e-16,-1.768133e-16,-3.083953e-16,7.735582e-17,-9.560254e-17,1.953170e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.053315e+00,-2.198333e+00,-7.299344e-01,-1.844051e+00,-1.451096e+00,-1.871317e+00,-2.113536e+00,-1.781806e+00,-1.288073e+00
25%,-7.864170e-01,-2.032073e-01,-7.299344e-01,-6.780234e-01,-7.898373e-01,-6.968193e-01,-7.585054e-01,-7.124846e-01,-7.872573e-01
50%,-2.822754e-01,2.955743e-01,-7.299344e-01,2.159310e-02,-1.285782e-01,-9.849036e-02,1.993608e-01,-1.778241e-01,-2.700210e-01
75%,4.220401e-01,7.943558e-01,1.344616e+00,7.212096e-01,5.326810e-01,6.106402e-01,7.016564e-01,8.380308e-01,4.032071e-01
max,3.958445e+00,1.293137e+00,1.344616e+00,1.887237e+00,2.516458e+00,1.940260e+00,1.671204e+00,2.067750e+00,2.718455e+00


In [15]:
from scipy.stats import zscore
df_zscore2 = df.apply(zscore,axis=0)
df_zscore2.describe()

,AGE,MONTH,SEX,HEADLEN,HEADWTH,NECK,LENGTH,CHEST,WEIGHT
count,5.400000e+01,5.400000e+01,5.400000e+01,5.400000e+01,5.400000e+01,5.400000e+01,5.400000e+01,5.400000e+01,5.400000e+01
mean,-1.233581e-17,3.577385e-16,-1.233581e-17,-2.117648e-16,-1.027984e-16,-2.446603e-16,3.340949e-17,-2.158767e-17,8.223874e-18
std,1.009390e+00,1.009390e+00,1.009390e+00,1.009390e+00,1.009390e+00,1.009390e+00,1.009390e+00,1.009390e+00,1.009390e+00
min,-1.063206e+00,-2.218976e+00,-7.367884e-01,-1.861366e+00,-1.464722e+00,-1.888888e+00,-2.133381e+00,-1.798537e+00,-1.300168e+00
25%,-7.938013e-01,-2.051154e-01,-7.367884e-01,-6.843899e-01,-7.972538e-01,-7.033623e-01,-7.656277e-01,-7.191748e-01,-7.946495e-01
50%,-2.849259e-01,2.983497e-01,-7.367884e-01,2.179586e-02,-1.297855e-01,-9.941517e-02,2.012327e-01,-1.794939e-01,-2.725565e-01
75%,4.260030e-01,8.018147e-01,1.357242e+00,7.279816e-01,5.376828e-01,6.163741e-01,7.082449e-01,8.458998e-01,4.069932e-01
max,3.995615e+00,1.305280e+00,1.357242e+00,1.904958e+00,2.540088e+00,1.958479e+00,1.686896e+00,2.087166e+00,2.743981e+00


+ Train a $K$-NN model with default parameters, using the raw dataset and another using the normalized dataset
+ Compare their results using an appropriate procedure and suitable measures+ Compare their performances 

In [28]:
X = df_zscore2.copy()
y = df_zscore2['SEX']
del X['SEX']

X_train, X_test, y_train, y_test = train(X,y, test_size = 0.33, random_state =10)

dt  = KNN()

dt.fit(X_train, y_train) #entrainement du DecisionTree

ValueError: Unknown label type: 'continuous'

In [25]:
y_predict = dt.predict(X_test)
pds.crosstab(y_test, y_predict) #matrice de contingence - lignes sont les vraies valeurs

NotFittedError: Must fit neighbors before querying.

+ Train and test different $K$-NN models to classify the bears in two categories ("males" and "females")
    + Using different number of neighbors.
    + And using different [weighting techniques](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).
    + Using different [distance metrics](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html).
+ Evaluate and compare the results using an appropriate procedure and suitable measures

__Bonus__
+ Train three learners to classify the bears in two categories ("males" and "females") using:
    + Decision Tree Classifier
    + A Linear SVM Classifier
    + A K-NN Classifier
+ Compare their results using an appropriate procedure and suitable measures

## Unsupervised Learning
+ Use DBSCAN to identify possible outliers?

+ Use Seaborn's Clustermap visualization to represent the dataset

+ Add [colored labels](https://seaborn.pydata.org/generated/seaborn.clustermap.html) representing the bear's sex

+ Include other possibly useful visualizations
+ You can compute a correlation matrix, and exclude correlated feature from the analysis

+ Given the Clustermap visualization, could you choose a smaller subset of features to conduct the analysis?
+ You can also create new features (e.g., Body mass index $BMI = \frac{Weight}{Size^2}$)

+ Cluster the dataset using the $K$-means algorithm
+ Choose an appropriate number of clusters

+ Build a white-box model to explain $a$ $posteriori$ the clustering model
+ Analyze the clustering model